# Car Market Analysis


## 1. Project Overview

## 2. Setup and Imports

In [ ]:
import pandas as pd # For data manipulation and analysis, especially for working with DataFrames
import numpy as np # For numerical computations and array manipulations
import matplotlib.pyplot as plt # For creating static, interactive, and animated visualizations
import seaborn as sns # For advanced data visualization, e.g. with built-in datasets

pd.options.display.max_columns = None # Specifies that Pandas should display all columns, helpful for EDA
pd.options.display.width = 120 # Controls how wide Pandas displays the DataFrame before it wraps rows or hides columns
sns.set_theme(style="whitegrid") # Sets a global visual style for all Seaborn plots (white background, subtle grid lines)


## 3. Data Loading

## 4. Data Cleaning

## 5. Feature Engineering

## 6. Exploratory Data Analysis

## 7. Visual Analysis

## 8. Correlation Analysis

## 9. Key Insights and Conclusion